<a href="https://colab.research.google.com/github/AnnaVeller/detect-license-plates/blob/master/Detect_license_plates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install asyncio

In [ ]:
!pip install cython
!pip install matplotlib
!pip install setuptools
!pip install numpy
!pip install Keras==2.2.*
!pip install imgaug
#!pip install tensorflow==1.15.2
!pip install opencv_python
!pip install imutils
!pip install scikit_image
!pip install jupyter

!pip install GitPython
!pip install pycocotools
!pip install tqdm

In [ ]:
!pip install Nomeroff-net-gpu

In [ ]:
!pip install mrcnn

In [ ]:
!pip install tensorflow-gpu==1.15.2

In [ ]:
import os
import numpy as np
import sys
import matplotlib.image as mpimg
from matplotlib import pyplot as plt

In [ ]:
NOMEROFF_NET_DIR = os.path.abspath("/content/test")
MASK_RCNN_DIR = os.path.join(NOMEROFF_NET_DIR, 'Mask_RCNN')
MASK_RCNN_LOG_DIR = os.path.join(NOMEROFF_NET_DIR, 'logs')
sys.path.append(NOMEROFF_NET_DIR)
from NomeroffNet import filters, RectDetector, TextDetector, OptionsDetector, Detector, textPostprocessing, textPostprocessingAsync

# Initialize npdetector with default configuration file.
nnet = Detector(MASK_RCNN_DIR, MASK_RCNN_LOG_DIR)
nnet.loadModel("latest")

rectDetector = RectDetector()

optionsDetector = OptionsDetector()
optionsDetector.load("latest")

# Initialize text detector.
textDetector = TextDetector.get_static_module("ru")()
textDetector.load("latest")

In [ ]:
files = os.listdir("/content/test2")
files.sort()
names = list(map(lambda x: os.path.splitext(x)[0], files))
pic = list(map(lambda x: "/content/test2/" + x, files))
print(pic)
print(names)

In [ ]:
right_detect = 0
for i in range(len(pic)):
    img_path = pic[i]
    # Detect numberplate
    img = mpimg.imread(img_path)
    NP = nnet.detect([img])

    # Generate image mask.
    cv_img_masks = filters.cv_img_mask(NP)

    # Detect points.
    arrPoints = rectDetector.detect(cv_img_masks)
    zones = rectDetector.get_cv_zonesBGR(img, arrPoints)

    # find standart
    regionIds, stateIds, countLines = optionsDetector.predict(zones)
    regionNames = optionsDetector.getRegionLabels(regionIds)

    # find text with postprocessing by standart
    textArr = textDetector.predict(zones)
    textArr = textPostprocessing(textArr, regionNames)
    state = False
    for k in range(len(textArr)):
        if names[i] == textArr[k]:
            state = True
            right_detect = right_detect + 1
    plt.axis("off")
    plt.imshow(img)
    plt.show()
    #print(arrPoints)
    for zone, points in zip(zones, arrPoints):
            plt.axis("off")
            plt.imshow(zone)
            plt.show()  
    print("real: ", names[i])
    print("predict: ", textArr, state)
    print('\n\n')

In [62]:
print(" right detect  | %d\n ---------------------\n all detect    | %d\n ---------------------\n right percent | %f"
%(right_detect,len(pic),right_detect/len(pic)*100),"%") 

 right detect  | 12
 ---------------------
 all detect    | 34
 ---------------------
 right percent | 35.294118 %
